In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import joblib

In [36]:
from sklearn.svm import SVC

In [2]:
from sklearn.model_selection import GridSearchCV

In [37]:
train_df = pd.read_csv('C:/Users/hp/Downloads/train.csv')


In [38]:
train_df.isnull().sum()

Embeddings    0
Labels        0
dtype: int64

In [39]:
valid_df = pd.read_csv('C:/Users/hp/Downloads/validation.csv')
valid_df.head()

,Embeddings,Labels
0,"[0.5469580863489887, 0.4764355187703884, 0.429...",1
1,"[0.4557014388234454, 0.42140761348218225, 0.39...",1
2,"[0.5709403438092233, 0.37141592889673375, 0.45...",0
3,"[0.5157037219388774, 0.5018736842637056, 0.463...",1
4,"[0.46181101920528106, 0.37636353034704323, 0.5...",1


In [6]:
test_df = pd.read_csv('C:/Users/hp/Downloads/test.csv')
test_df.head()

,Embeddings
0,"[0.3337063161257903, 0.5192625182470981, 0.484..."
1,"[0.5360962959480485, 0.6644046340186954, 0.537..."
2,"[0.5753105821255707, 0.41494639938382866, 0.41..."
3,"[0.3998871034260141, 0.4398297367391118, 0.447..."
4,"[0.4351530915561264, 0.5057971600970778, 0.584..."


In [61]:
def convert_string_to_array(s):
    s = s.replace('\x00', '0')  # Remove null bytes
    try:
        return np.array(eval(s))
    except Exception as e:
        print(f"Error parsing string: {s}. Error: {e}")
        return np.zeros(768)  # Return None if parsing fails

In [62]:
X_train = np.array([convert_string_to_array(row) for row in train_df.iloc[:, :-1].values.flatten()])
y_train = train_df.iloc[:, -1].astype(float).values
X_valid = np.array([convert_string_to_array(row) for row in valid_df.iloc[:, :-1].values.flatten()])
y_valid = valid_df.iloc[:, -1].astype(float).values
X_test = np.array([convert_string_to_array(row) for row in test_df.values.flatten()])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

#save the scaler
joblib.dump(scaler, 'F:/summary-model/scaler.pkl')

['F:/summary-model/scaler.pkl']

In [63]:
from sklearn.model_selection import cross_val_score

In [82]:
svm_clf = SVC(C=1.0, kernel='rbf', gamma='scale')

# Evaluate the model using cross-validation
cv_scores = cross_val_score(svm_clf, X_train, y_train, cv=5)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean cross-validation score: {cv_scores.mean()}")

Cross-validation scores: [0.61122575 0.59925712 0.61246389 0.61782914 0.61932287]
Mean cross-validation score: 0.6120197542611356


In [83]:
svm_clf.fit(X_train, y_train)
joblib.dump(svm_clf, 'F:/summary-model/svm_model.pkl')

['F:/summary-model/svm_model.pkl']

In [81]:
train_accuracy = accuracy_score(y_train, svm_clf.predict(X_train))
valid_accuracy = accuracy_score(y_valid, svm_clf.predict(X_valid))

print(f'SVM Train Accuracy: {train_accuracy}')
print(f'SVM Validation Accuracy: {valid_accuracy}')

SVM Train Accuracy: 0.9248803037807496
SVM Validation Accuracy: 0.6173333333333333


In [67]:
test_predictions = svm_clf.predict(X_test)

# Save predictions to a CSV file
output_df = pd.DataFrame(test_predictions)
new_df=test_df.merge(output_df,left_index=True,right_index=True
)
new_df.columns=['Embedded','Labels']
new_df.to_csv('F:/summary-model/test_predictions.csv',index=False)

In [73]:
test_data=pd.read_csv('F:/summary-model/test_predictions.csv')
test_data.head()

,Embedded,Labels
0,"[0.3337063161257903, 0.5192625182470981, 0.484...",-1.0
1,"[0.5360962959480485, 0.6644046340186954, 0.537...",-1.0
2,"[0.5753105821255707, 0.41494639938382866, 0.41...",0.0
3,"[0.3998871034260141, 0.4398297367391118, 0.447...",1.0
4,"[0.4351530915561264, 0.5057971600970778, 0.584...",1.0
